# EBAC - Regressão II - regressão múltipla

## Tarefa II

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [29]:
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,  StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score, mean_squared_error


%matplotlib inline

In [18]:
df = pd.read_csv('previsao_de_renda.csv')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [20]:
# Exibindo todas as colunas disponíveis no DataFrame
print(df.columns)

# Criando variáveis dummy para colunas categóricas
df_dummies = pd.get_dummies(df[['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']], drop_first=True)

# Concatenando as variáveis dummy com as colunas numéricas
df = pd.concat([df[['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'renda']], df_dummies], axis=1)

X = df.drop('renda', axis=1)
y = df['renda']

# Separando os dados em treinamento e teste (75% para treinamento, 25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Imprimindo o tamanho dos conjuntos de treinamento e teste
print("Tamanho do conjunto de treinamento:", len(X_train))
print("Tamanho do conjunto de teste:", len(X_test))

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')
Tamanho do conjunto de treinamento: 11250
Tamanho do conjunto de teste: 3750


In [21]:
# Imputando valores ausentes
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Inicializando listas para armazenar os resultados
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_scores = []

# Iterar sobre os valores de alpha
for alpha in alphas:
    model = Ridge(alpha=alpha)
    model.fit(X_train_imputed, y_train)
    y_pred = model.predict(X_test_imputed)
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Encontrando o melhor modelo (maior R^2)
best_alpha = alphas[r2_scores.index(max(r2_scores))]
best_r2 = max(r2_scores)

# Imprimindo os resultados
print(f"R^2 para diferentes valores de alpha: {dict(zip(alphas, r2_scores))}")
print(f"Melhor modelo - Alpha: {best_alpha}, R^2: {best_r2}")

R^2 para diferentes valores de alpha: {0: 0.2691159056847583, 0.001: 0.2691159145559313, 0.005: 0.2691159500045429, 0.01: 0.26911599423430155, 0.05: 0.2691163448655334, 0.1: 0.26911677529614875}
Melhor modelo - Alpha: 0.1, R^2: 0.26911677529614875


Considerando o R2, que mede a proporção da variabilidade na variável dependente que é explicada pelas variáveis independentes no modelo está mais próxima de 1, temos esse modelo como sendo melhor.

In [25]:
# Imputando valores NaN
imputer = SimpleImputer(strategy='mean')
df[['posse_de_imovel', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'sexo_M', 'tipo_renda_Empresário', 'educacao_Superior completo']] = imputer.fit_transform(df[['posse_de_imovel', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'sexo_M', 'tipo_renda_Empresário', 'educacao_Superior completo']])

# Separando os dados em treinamento e teste (75% para treinamento, 25% para teste)
X = df[['posse_de_imovel', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'sexo_M', 'tipo_renda_Empresário', 'educacao_Superior completo']]
y = df['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Regressão LASSO
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
best_model_lasso = None
best_r2_lasso = -float('inf')

for alpha in alphas:
    # Criar e treinar o modelo LASSO
    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    # Fazendo previsões no conjunto de testes
    y_pred = model.predict(X_test)

    # Avaliando o desempenho usando R2
    r2 = r2_score(y_test, y_pred)

    # Atualizando o melhor modelo se necessário
    if r2 > best_r2_lasso:
        best_r2_lasso = r2
        best_model_lasso = model

print(f"Melhor modelo LASSO (R2 = {best_r2_lasso}): {best_model_lasso}")

Melhor modelo LASSO (R2 = 0.26525661212275686): Lasso(alpha=0)


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.871e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

In [26]:
def forward_selection(X, y, significance_level=0.05):
    included = []
    while True:
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)

        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]

        min_p_value = new_pval.min()
        if min_p_value < significance_level:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
        else:
            break

    return included

# Realizando seleção de variáveis forward
selected_features_forward = forward_selection(X_train, y_train)

# Treinando o modelo com as variáveis selecionadas
X_train_forward = X_train[selected_features_forward]
model_forward = sm.OLS(y_train, sm.add_constant(X_train_forward)).fit()

# Fazendo previsões no conjunto de testes
X_test_forward = X_test[selected_features_forward]
y_pred_forward = model_forward.predict(sm.add_constant(X_test_forward))

# Avaliando o desempenho usando R2
r2_forward = r2_score(y_test, y_pred_forward)

print(f"Variáveis selecionadas (forward): {selected_features_forward}")
print(f"R2 do modelo (forward): {r2_forward}")
print(model_forward.summary())

Variáveis selecionadas (forward): ['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'educacao_Superior completo']
R2 do modelo (forward): 0.26361736232790967
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     967.0
Date:                Wed, 31 Jan 2024   Prob (F-statistic):               0.00
Time:                        17:15:27   Log-Likelihood:            -1.1580e+05
No. Observations:               11250   AIC:                         2.316e+05
Df Residuals:                   11245   BIC:                         2.316e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err 

Melhor modelo LASSO (R2 = 0.26525661212275686). Inclusive, até o momento o modelo LASSO é o melhor de todos.

In [30]:
# Criar e treinar uma árvore de decisão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

# Fazer previsões no conjunto de testes
y_pred_tree = tree_model.predict(X_test)

# Avaliar o desempenho usando R²
r2_tree = r2_score(y_test, y_pred_tree)

print(f"R2 do modelo de árvore de decisão: {r2_tree}")

R2 do modelo de árvore de decisão: 0.30476532010427015


In [ ]:
Verifica-se uma melhora sensível no modelo.